# node_union_df
0. create node-level dataframe from combined graphs
1. out = node_union_df.csv

** to do: 3 dataframes (by sentiment) **

In [3]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os
from glob import glob
import re

pd.set_option('display.mpl_style', 'default') 
pd.set_option('display.width', 5000) 
pd.set_option('display.max_columns', 60)

# use a list comprehension to create a list that matches files in a directory using regular expressions
# http://stackoverflow.com/questions/2225564/get-a-filtered-list-of-files-in-a-directory
#gml_files = [f for f in os.listdir('.') if re.match(r'(pos|neg|neu)_u*all\.gml', f)]

gml_files = [f for f in os.listdir('.') if re.match(r'(pos|neg|neu)_uall\.gml', f)]


def calculate_graph_inf(graph):
    graph.name = filename
    info = nx.info(graph)
    print info
    #plt.figure(figsize=(10,10))
    #nx.draw_spring(graph, arrows=True, with_labels=True)

def highest_centrality(cent_dict):
    """Returns a tuple (node,value) with the node
    with largest value from centrality dictionary."""
    # create ordered tuple of centrality data
    cent_items = [(b,a) for (a,b) in cent_dict.iteritems()]
    # sort in descending order
    cent_items.sort()
    cent_items.reverse()
    return tuple(reversed(cent_items[0]))

In [4]:
gml_files.sort()
gml_files

['neg_uall.gml', 'neu_uall.gml', 'pos_uall.gml']

In [5]:
# create empty dataframe with columns
data_columns = ['name',
                'sentiment'
                ]
data = pd.DataFrame(columns = data_columns)
combined_df = pd.DataFrame()

In [7]:
for graph_num, gml_graph in enumerate(gml_files):
    graph = nx.read_gml(gml_graph)
    (filepath, filename) = os.path.split(gml_graph)
    print('-' * 40)
    print(gml_graph)
    calculate_graph_inf(graph)

    # calculate variables and save into list
    sent = filename.split('_')[0]
    deg_cent = nx.degree_centrality(graph)
    bet_cent = nx.betweenness_centrality(graph)
    clo_cent = nx.closeness_centrality(graph)
    
    graph_values = {'name':filename,
                    'sentiment':sent,
                    }

    data = data.append(graph_values, ignore_index=True)

    #
    degree = nx.degree(graph)
    deg_df = pd.DataFrame.from_dict(degree, orient = 'index')
    deg_df.columns = ['degree']
    # degree centrality
    deg_cent = nx.degree_centrality(graph)
    dc_df = pd.DataFrame.from_dict(deg_cent, orient = 'index')
    dc_df.columns = ['degree centrality']
    # betweenness centrality
    bet_cent = nx.betweenness_centrality(graph)
    bc_df = pd.DataFrame.from_dict(bet_cent, orient = 'index')
    bc_df.columns = ['betweenness centrality']
    # closeness centrality
    clo_cent = nx.closeness_centrality(graph)
    cc_df = pd.DataFrame.from_dict(clo_cent, orient = 'index')
    cc_df.columns = ['closeness centrality']
    
    # concat node frames into node_df
    frames = [deg_df, dc_df, bc_df, cc_df]
    node_df = pd.concat(frames, axis = 1)
    node_df.index.name = 'node'
    node_df = node_df.reset_index()
    
    values = pd.DataFrame(graph_values, columns = ('name', 'sentiment'), index = [0])
    
    # df = merges graph_values with node_df for single graph and fill NaNs
    df = pd.concat([values, node_df], axis = 1)
    df = df.fillna(method='ffill')
    
    # append to combined_df
    combined_df = combined_df.append(df)
    
    
#    if graph_num == 0:
#        break

----------------------------------------
neg_uall.gml
Name: neg_uall.gml
Type: MultiGraph
Number of nodes: 1563
Number of edges: 1870
Average degree:   2.3928
----------------------------------------
neu_uall.gml
Name: neu_uall.gml
Type: MultiGraph
Number of nodes: 250
Number of edges: 238
Average degree:   1.9040
----------------------------------------
pos_uall.gml
Name: pos_uall.gml
Type: MultiGraph
Number of nodes: 948
Number of edges: 1110
Average degree:   2.3418


In [8]:
combined_df

,name,sentiment,node,degree,degree centrality,betweenness centrality,closeness centrality
0,neg_uall.gml,neg,religious exemption,1,0.000640,0.000000,0.137777
1,neg_uall.gml,neg,adverse effects,15,0.009603,0.011682,0.122292
2,neg_uall.gml,neg,flu shot campaign,1,0.000640,0.000000,0.002459
3,neg_uall.gml,neg,vaccine program,1,0.000640,0.000000,0.140920
4,neg_uall.gml,neg,government health agencies,1,0.000640,0.000000,0.001152
5,neg_uall.gml,neg,controversial,1,0.000640,0.000000,0.155737
6,neg_uall.gml,neg,swine-flu-related drugs,1,0.000640,0.000000,0.000640
7,neg_uall.gml,neg,safe level of exposure,1,0.000640,0.000000,0.166641
8,neg_uall.gml,neg,heightened emotion,2,0.001280,0.000973,0.144974
9,neg_uall.gml,neg,Dr. Paul Offit,16,0.010243,0.015700,0.155415


In [6]:
# save dataframe to csv
combined_df.to_csv('node_union_df', encoding = 'utf-8')

- - -

In [ ]:
# df = pd.read_csv('df')
# df